<a href="https://colab.research.google.com/github/1PD-IS-NO-1/mlproject/blob/main/attendence_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
student_image_dir = '/content/gdrive/MyDrive/student_image'
for filename in os.listdir(student_image_dir):
    print(filename)

Mounted at /content/gdrive
12.jpeg
6.jpeg
17.jpeg
19.jpeg
24.jpeg
13.jpeg
14.jpeg
16.jpeg
9.jpeg
23.jpeg
18.jpeg
22.jpeg
11.jpeg
10.jpeg
5.jpeg
21.jpeg
20.jpeg
25.jpeg


In [ ]:
!install  torchvision-cpu

install: missing destination file operand after 'torchvision-cpu'
Try 'install --help' for more information.


In [ ]:
!pip install face_recognition Pillow torch torchvision

In [ ]:
import os
import csv
import datetime
import logging
from typing import List, Tuple, Optional

import torch
from torch import nn
from torchvision import transforms, models
from PIL import Image
import face_recognition
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class AttendanceSystem:
    def __init__(self, database_dir: str):
        self.database_dir = database_dir
        self.database = self._load_database()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def _load_database(self) -> dict:
        database = {}
        print("database encoding is start")
        for filename in os.listdir(self.database_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                roll_number = os.path.splitext(filename)[0]
                image_path = os.path.join(self.database_dir, filename)
                try:
                    image = face_recognition.load_image_file(image_path)
                    encodings = face_recognition.face_encodings(image)
                    if encodings:
                        database[roll_number] = encodings[0]
                    else:
                        logging.warning(f"No face found in {filename}. Skipping.")
                except Exception as e:
                    logging.error(f"Error processing {filename}: {str(e)}")
        return database

    def _load_model(self) -> nn.Module:
        model = models.resnet50(pretrained=True)
        model = model.to(self.device)
        model.eval()
        return model

    def process_classroom_image(self, image_path: str) -> List[Tuple[str, float]]:
        print("working on classroom image now")
        try:
            classroom_image = face_recognition.load_image_file(image_path)
        except Exception as e:
            logging.error(f"Error loading classroom image: {str(e)}")
            return []

        face_locations = face_recognition.face_locations(classroom_image)
        if not face_locations:
            logging.warning("No faces detected in the classroom image.")
            return []

        face_encodings = face_recognition.face_encodings(classroom_image, face_locations)

        matches = []
        for face_encoding in face_encodings:
            match = self._find_best_match(face_encoding)
            if match:
                matches.append(match)

        return matches

    def _find_best_match(self, face_encoding: np.ndarray) -> Optional[Tuple[str, float]]:
        print("doing mathcing")
        best_match = None
        best_distance = float('inf')
        for roll_number, known_encoding in self.database.items():
            distance = face_recognition.face_distance([known_encoding], face_encoding)[0]
            if distance < best_distance:
                best_distance = distance
                best_match = (roll_number, distance)

        if best_match and best_match[1] < 0.6:  # Adjust this threshold as needed
            return best_match
        return None

    def record_attendance(self, course: str, date: str, classroom_image_path: str) -> List[str]:
        matches = self.process_classroom_image(classroom_image_path)

        csv_file_path = f'{course}_attendance.csv'
        if not os.path.exists(csv_file_path):
            with open(csv_file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Date', 'Roll Number', 'Confidence'])

        present_students = []
        with open(csv_file_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for roll_number, distance in matches:
                confidence = 1 - distance
                writer.writerow([date, roll_number, f"{confidence:.2f}"])
                present_students.append(roll_number)

        logging.info(f"Recorded attendance for {len(present_students)} students in {course} on {date}")
        return present_students

    def generate_report(self, course: str, start_date: str, end_date: str) -> None:
        csv_file_path = f'{course}_attendance.csv'
        report_file_path = f'{course}_attendance_report.csv'

        if not os.path.exists(csv_file_path):
            logging.error(f"No attendance records found for {course}")
            return

        attendance_data = {}
        with open(csv_file_path, 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # Skip header
            for row in reader:
                date, roll_number, _ = row
                if start_date <= date <= end_date:
                    if roll_number not in attendance_data:
                        attendance_data[roll_number] = set()
                    attendance_data[roll_number].add(date)

        with open(report_file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Roll Number', 'Attendance Percentage', 'Days Present', 'Total Days'])

            start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
            end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
            total_days = (end - start).days + 1

            for roll_number, dates_present in attendance_data.items():
                attendance_percentage = (len(dates_present) / total_days) * 100
                writer.writerow([roll_number, f"{attendance_percentage:.2f}%", len(dates_present), total_days])

        logging.info(f"Generated attendance report for {course} from {start_date} to {end_date}")

# Example usage
if __name__ == "__main__":
    database_dir = '/content/gdrive/MyDrive/student_image'

    attendance_system = AttendanceSystem(database_dir)
    subject  = input("enther the subject name")
    # Record attendance
    present_students = attendance_system.record_attendance(subject, '2024-03-20', '/content/test1.jpg')
    print(f"Present students: {present_students}")

    # Generate report
    attendance_system.generate_report(subject, '2024-03-01', '2024-03-31')

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-5z8y1whm/dlib_58415f25543c4f6dadb365c4be81f21f/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
!pip uninstall -y face_recognition
!pip install face_recognition-cpu

Found existing installation: face-recognition 1.3.0
Uninstalling face-recognition-1.3.0:
  Successfully uninstalled face-recognition-1.3.0
ERROR: Could not find a version that satisfies the requirement face_recognition-cpu (from versions: none)
ERROR: No matching distribution found for face_recognition-cpu


In [ ]:
import torch
print("CUDA version:", torch.version.cuda)
print("Is CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))

CUDA version: 12.1
Is CUDA available: False


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import csv
import os

def record_attendance_new(present_students: list, subject: str):
    date = input("Enter the date (YYYY-MM-DD): ")

    # Create or open the CSV file based on the subject
    csv_file_path = f'{subject}.csv'

    # Check if the file exists
    if not os.path.exists(csv_file_path):
        # If the file doesn't exist, create it and add the date as the first column header
        with open(csv_file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Roll Number', date])  # Initialize with Roll Number as the first column
            for roll_number in present_students:
                writer.writerow([roll_number])  # Insert the roll numbers under the date column
    else:
        # If the file exists, open it and check the structure
        # Read the existing CSV file
        with open(csv_file_path, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            rows = list(reader)

        # Check if the date column already exists
        if date not in rows[0]:
            # Add a new date column if it doesn't exist
            rows[0].append(date)

        # Collect the index of the date column
        date_col_index = rows[0].index(date)

        # Update or add roll numbers for the given date
        for roll_number in present_students:
            # Check if the roll number already exists in the file
            found = False
            for row in rows[1:]:
                if row[0] == str(roll_number):  # Roll number matches
                    found = True
                    # If the roll number is found and the corresponding date cell is empty, fill it
                    if len(row) < date_col_index + 1:  # Extend the row if it doesn't have enough columns
                        row.extend([''] * (date_col_index - len(row) + 1))
                    if row[date_col_index] == '':
                        row[date_col_index] = 'Present'
                    break
            if not found:
                # If the roll number is not found, add a new row for this roll number
                new_row = [str(roll_number)] + [''] * (date_col_index - 1) + ['Present']
                rows.append(new_row)

        # Write the updated data back to the CSV file
        with open(csv_file_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(rows)

    print(f"Attendance recorded for {len(present_students)} students on {date} in {subject}.")
record_attendence_new()

In [ ]:
!pip install weaviate-client torch transformers pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully

In [ ]:
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import weaviate
import numpy as np

# Initialize CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Connect to Weaviate (run Weaviate locally or connect to a cloud instance)
client = weaviate.Client("http://localhost:8080")

# Create a schema in Weaviate for storing student images
class_obj = {
    "class": "StudentImage",
    "vectorizer": "none",  # We will provide custom embeddings from CLIP
    "properties": [
        {"name": "roll_number", "dataType": ["string"]},
        {"name": "image_path", "dataType": ["string"]},
        {"name": "embedding", "dataType": ["number[]"]},
    ]
}

# Create the class in Weaviate (if it doesn't already exist)
client.schema.delete_class("StudentImage")  # Optional: Delete class before recreating
client.schema.create_class(class_obj)

# Function to convert an image to CLIP embeddings
def get_image_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    return embedding.squeeze().numpy()  # Convert torch tensor to numpy array

# Convert all images in the folder into embeddings and store them in Weaviate
def store_images_to_weaviate(folder_path):
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            roll_number = os.path.splitext(image_file)[0]  # Use image file name as roll number

            # Get image embedding
            embedding = get_image_embedding(image_path)

            # Store the image data into Weaviate
            data_object = {
                "roll_number": roll_number,
                "image_path": image_path,
                "embedding": embedding.tolist()
            }
            client.data_object.create(data_object, class_name="StudentImage")

# Folder path containing student images (each named as the roll number)
folder_path = 'content/student_image'

# Convert images and store in Weaviate
store_images_to_weaviate(folder_path)
